In [ ]:
!pip install diffusers==0.20.2 transformers accelerate

In [ ]:
import torch
import requests
from PIL import Image
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler

# Load the pipeline
pipeline = DiffusionPipeline.from_pretrained(
    "sudo-ai/zero123plus-v1.1", custom_pipeline="sudo-ai/zero123plus-pipeline",
    torch_dtype=torch.float16
)
# Feel free to tune the scheduler
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
    pipeline.scheduler.config, timestep_spacing='trailing'
)
pipeline.to('cuda:0')
# Run the pipeline


In [ ]:
import os
import torch
from PIL import Image
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler

def process_images(input_folder, output_folder, model_version):
    # Load the pipeline
    pipeline = DiffusionPipeline.from_pretrained(
        f"sudo-ai/{model_version}", custom_pipeline="sudo-ai/zero123plus-pipeline",
        torch_dtype=torch.float16
    )
    # Tune the scheduler
    pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
        pipeline.scheduler.config, timestep_spacing='trailing'
    )
    pipeline.to('cuda:0')
    
    # Walk through the input folder
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file)
                
                cond = Image.open(file_path)
                
                # Run the pipeline
                result = pipeline(cond, num_inference_steps=150).images[0]
                
                # Determine the relative path and the corresponding output path
                relative_path = os.path.relpath(file_path, input_folder)
                output_path = os.path.join(output_folder, relative_path)
                print(output_path)
                # Ensure the output directory exists
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                
                # Save the output image
                result.save(output_path)

# Define input and output folders
input_folder = "output_forground"
output_folder_v1_2 = "v1.2"
output_folder_v1_1 = "v1.1"

# Process images for both model versions
process_images(input_folder, output_folder_v1_2, "zero123plus-v1.2")
process_images(input_folder, output_folder_v1_1, "zero123plus-v1.1")
